In [1]:
#https://peaceful0907.medium.com/%E4%BE%86%E7%8E%A9%E9%BB%9Enlp-lstm-vs-bert-on-imdb-dataset-4aa18ecd65e2
import os
import re
from itertools import chain

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#%cd drive/MyDrive/二技資管一甲/下學期/news_dataset/
! ls

drive  sample_data


In [11]:
#files = os.listdir("drive/MyDrive/二技資管一甲/下學期/news_dataset/")
#print(files)

In [12]:
#news_data = pd.read_json("drive/MyDrive/二技資管一甲/下學期/news_dataset/News_Category_Dataset_v2.json", lines = True)

TAG_RE = re.compile(r'<[^>]+>')
def preprocess_text(sen):
    # Removing html tags
    sentence = TAG_RE.sub('', sen)
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

def readIMDB(path, seg):
    classes = ['pos', 'neg']
    data = []
    for label in classes:
        files = os.listdir("drive/MyDrive/二技資管一甲/下學期/news_dataset/")
        for file in files:
            with open('aclImdb_v1.tar', 'r', encoding='utf8') as rf:
                review = rf.read().replace('\n', '')
                if label == 'pos':
                    data.append([preprocess_text(review), 1])
                elif label == 'neg':
                    data.append([preprocess_text(review), 0])
    return data

def tokenizer(text):
    return [tok.lower() for tok in text.split(' ')] #簡單使用空格來斷詞

train_data = readIMDB('aclImdb_v1.tar','train')
test_data = readIMDB('aclImdb_v1.tar', 'test')

train_tokenized = []
test_tokenized = []
for review, score in train_data:
    train_tokenized.append(tokenizer(review))
for review, score in test_data:
    test_tokenized.append(tokenizer(review))

vocab = set(chain(*train_tokenized)) #把tokenized 所有字給串起來
vocab_size = len(vocab)

FileNotFoundError: ignored

In [ ]:
### let us see some preprocess result ###
text = "an apple a day, keeps the doctor away"
token = tokenizer(text)
print('text: '+text)
print('token: '+ str(token))
print('encode to index: '+str(encode_samples([token])))
text: an apple a day, keeps the doctor away
token: ['an', 'apple', 'a', 'day,', 'keeps', 'the', 'doctor', 'away']
encode to index: [[23314, 28267, 26943, 0, 49709, 27809, 29103, 70204]]

In [ ]:
### load word2vec model ###
#pre-train model download from: https://github.com/stanfordnlp/GloVe
#preprocess:https://stackoverflow.com/questions/51323344/cant-load-glove-6b-300d-txt
wvmodel = gensim.models.KeyedVectors.load_word2vec_format('glove.6B.100d.w2vformat.txt',binary=False, encoding='utf-8')

## map golve pretrain weight to pytorch embedding pretrain weight
embed_size = 100
weight = torch.zeros(vocab_size+1, embed_size) #given 0 if the word is not in glove
for i in range(len(wvmodel.index2word)):
    try:
        index = word_to_idx[wvmodel.index2word[i]] #transfer to our word2ind
    except:
        continue
    weight[index, :] = torch.from_numpy(wvmodel.get_vector(wvmodel.index2word[i]))

In [ ]:
### build model ###
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 bidirectional, weight, labels, **kwargs):
        super(RNN, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=self.num_hiddens,
                               num_layers=num_layers, bidirectional=self.bidirectional,
                               dropout=0.3)

        if self.bidirectional:
            self.linear1 = nn.Linear(num_hiddens * 4, labels)
        else:
            self.linear1 = nn.Linear(num_hiddens * 2, labels)



    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        states, hidden = self.encoder(embeddings.permute([1, 0, 2]))
        encoding = torch.cat([states[0], states[-1]], dim=1) #if it's bidirectional, choose first and last output
        outputs = self.linear1(encoding)

        return outputs

num_epochs = 10
num_hiddens = 100
num_layers = 2
bidirectional = True
labels = 2
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
net = RNN(vocab_size=(vocab_size+1), embed_size=embed_size,
                   num_hiddens=num_hiddens, num_layers=num_layers,
                   bidirectional=bidirectional, weight=weight,
                   labels=labels)

print(net)

In [ ]:
net.to(device)
loss_function = nn.CrossEntropyLoss() # ~ nn.LogSoftmax()+nn.NLLLoss()
optimizer = optim.Adam(net.parameters())

def train(net,num_epochs,loss_function,optimizer,train_iter,val_iter):
    for epoch in range(num_epochs):
        start = time.time()
        train_loss, val_losses = 0, 0
        train_acc, val_acc = 0, 0
        n, m = 0, 0
        net.train()
        for feature, label in train_iter:
            n += 1
            optimizer.zero_grad()
            feature = Variable(feature.to(device))
            label = Variable(label.to(device))

            score = net(feature)
            loss = loss_function(score, label)
            loss.backward()
            optimizer.step()
            train_acc += accuracy_score(torch.argmax(score.cpu().data,dim=1), label.cpu())
            train_loss += loss

        with torch.no_grad():
            net.eval()
            for val_feature, val_label in val_iter:
                m += 1
                val_feature = val_feature.to(device)
                val_label = val_label.to(device)
                val_score = net(val_feature)
                val_loss = loss_function(val_score, val_label)
                val_acc += accuracy_score(torch.argmax(val_score.cpu().data,dim=1), val_label.cpu())
                val_losses += val_loss

        runtime = time.time() - start
        print('epoch: %d, train loss: %.4f, train acc: %.2f, val loss: %.4f, val acc: %.2f, time: %.2f' %
              (epoch, train_loss.data/n, train_acc/n, val_losses.data/m, val_acc/m, runtime))

    #save final model
    state = {
            'epoch': epoch,
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict()
            }
    torch.save(state, os.path.join(model_save_path,'last_model.pt'))


def predict(net,test_iter):
    #state = torch.load(os.path.join(cwd,'checkpoint','epoch10_maxlen300_embed200.pt'),map_location=torch.device('cpu'))
    #net.load_state_dict(state['state_dict'])
    pred_list = []
    true_list = []
    softmax = nn.Softmax(dim=1)
    with torch.no_grad():
        net.eval()
        for batch,label in test_iter:
            output = net(batch.to(device))
            pred_list.extend(torch.argmax(softmax(output),dim=1).cpu().numpy())
            true_list.extend(label.cpu().numpy())

    acc = accuracy_score(pred_list, true_list)
    print('test acc: %f'%acc)

    return acc,pred_list,true_list


print('start to train...')
train(net,num_epochs,loss_function,optimizer,train_iter,val_iter)

print('start to predict test set...')
acc,pred_list,true_list = predict(net,test_iter)

print('Done')